Economic ChatBot

In [7]:
import os 
from langchain_openai import ChatOpenAI 
from langchain.tools import tool
from langchain.agents import AgentExecutor,create_openai_tools_agent 
import json 
from langchain import hub 
import requests 
from datetime import date

In [4]:
api_key = os.environ['OPENAI_API_KEY']
llm = ChatOpenAI(api_key=api_key)

In [5]:

map_indicator = {
 'CPI_for_All_Urban_Consumers_CPIU_Unadjusted':'CUUR0000SA0',
 'CPI_for_All_Urban_Consumers_CPI_1967_100_Unadjusted':'CUUR0000AA0',
 'CPI_for_Urban_Wage_Earners_and_Clerical_Workers_CPI_W_1982-84=100_Unadjusted':'CWUR0000SA0',
 'CPI_U_Less_Food_and_Energy_Unadjusted':'CUUR0000SA0L1E',
 'CPI_W_Less_Food_and_Energy_Unadjusted':'CWUR0000SA0L1E',
 'PPI_Final_Demand_Seasonally_Adjusted':'WPSFD4',
 'PPI_Final_Demand_Unadjusted':'WPUFD4',
 'PPI_Final_Demand_less_foods_and_energy_Unadjusted':'WPUFD49104',
 'Demand_less_foods_energy_and_trade_services_Unadjusted':'WPUFD49116',
 'PPI Finished Goods 1982=100 (Unadjusted)':'WPUFD49207',
 'Imports_All_Commodities':'EIUIR',
 'Exports_All_Commodities':'EIUIQ'

}

"""
map_indicator = {
 'CPI_for_All_Urban_Consumers_CPIU_Unadjusted':'CUUR0000SA0',
 'CPI_for_All_Urban_Consumers_CPI_1967_100_Unadjusted':'CUUR0000AA0',
 'CPI_for_Urban_Wage_Earners_and_Clerical_Workers_CPI_W_1982-84=100_Unadjusted':'CWUR0000SA0',
 'CPI_U_Less_Food_and_Energy_Unadjusted':'CUUR0000SA0L1E',
 'CPI_W_Less_Food_and_Energy_Unadjusted':'CWUR0000SA0L1E',
 'PPI_Final_Demand_Unadjusted':'WPUFD4',
 'PPI_Final_Demand_less_foods_and_energy_Unadjusted':'WPUFD49104',
 'Demand_less_foods_energy_and_trade_services_Unadjusted':'WPUFD49116',
 'PPI Finished Goods 1982=100 (Unadjusted)':'WPUFD49207',
 'Imports_All_Commodities':'EIUIR',
 'Exports_All_Commodities':'EIUIQ'

}
"""

"\nmap_indicator = {\n 'CPI_for_All_Urban_Consumers_CPIU_Unadjusted':'CUUR0000SA0',\n 'CPI_for_All_Urban_Consumers_CPI_1967_100_Unadjusted':'CUUR0000AA0',\n 'CPI_for_Urban_Wage_Earners_and_Clerical_Workers_CPI_W_1982-84=100_Unadjusted':'CWUR0000SA0',\n 'CPI_U_Less_Food_and_Energy_Unadjusted':'CUUR0000SA0L1E',\n 'CPI_W_Less_Food_and_Energy_Unadjusted':'CWUR0000SA0L1E',\n 'PPI_Final_Demand_Unadjusted':'WPUFD4',\n 'PPI_Final_Demand_less_foods_and_energy_Unadjusted':'WPUFD49104',\n 'Demand_less_foods_energy_and_trade_services_Unadjusted':'WPUFD49116',\n 'PPI Finished Goods 1982=100 (Unadjusted)':'WPUFD49207',\n 'Imports_All_Commodities':'EIUIR',\n 'Exports_All_Commodities':'EIUIQ'\n\n}\n"

In [8]:
url = 'https://api.bls.gov/publicAPI/v1/timeseries/data/'
prompt_template  = f"Your job is to answer users economic questions by referencing economic data. Note that todays date is {str(date.today())}.Provide the answer in a table"

In [9]:
prompt = hub.pull("hwchase17/openai-tools-agent")

c:\Users\lehne\anaconda3\envs\LangChain\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
prompt[0].prompt.template = prompt_template


In [11]:
prompt.pretty_print()

================================ System Message ================================

Your job is to answer users economic questions by referencing economic data. Note that todays date is 2025-05-08.Provide the answer in a table

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [22]:
@tool 
def load_indicator(indicator,start,end):
  """
    indicator: Which economic indicator you would like to pull historical data on. This can be one of:
    CPI_for_All_Urban_Consumers_CPIU_Unadjusted,
    CPI_for_All_Urban_Consumers_CPI_1967_100_Unadjusted,
    CPI_for_Urban_Wage_Earners_and_Clerical_Workers_CPI_W_1982-84=100_Unadjusted,
    CPI_U_Less_Food_and_Energy_Unadjusted,
    CPI_W_Less_Food_and_Energy_Unadjusted,
    PPI_Final_Demand_Seasonally_Adjusted,
    PPI_Final_Demand_Unadjusted,
    PPI_Final_Demand_less_foods_and_energy_Unadjusted,
    Demand_less_foods_energy_and_trade_services_Unadjusted,
    PPI Finished Goods 1982=100 (Unadjusted),
    Imports_All_Commodities or
    Exports_All_Commodities
    start_year: the start year of the date range over which you want to pull economic data
    end_year: the end year of the date range over which you want to pull economic data
  """
  data = json.dumps({"seriesid":[map_indicator.get(indicator)],"startyear":start,"endyear":end})
  headers = {"Content-type":"application/json"}
  response = requests.post(url=url,data=data,headers=headers)
  json_load = json.loads(response.text)
  lst = []
  for outer in json_load["Results"]["series"]:
    for inner in outer["data"]:
      lst.append(outer)
  return lst
  #return json_load

In [18]:
lst = load_indicator("CPI_for_All_Urban_Consumers_CPIU_Unadjusted","2016","2018")

In [ ]:
lst 

[{'seriesID': 'CUUR0000SA0',
  'data': [{'year': '2018',
    'period': 'M12',
    'periodName': 'December',
    'value': '251.233',
    'footnotes': [{}]},
   {'year': '2018',
    'period': 'M11',
    'periodName': 'November',
    'value': '252.038',
    'footnotes': [{}]},
   {'year': '2018',
    'period': 'M10',
    'periodName': 'October',
    'value': '252.885',
    'footnotes': [{}]},
   {'year': '2018',
    'period': 'M09',
    'periodName': 'September',
    'value': '252.439',
    'footnotes': [{}]},
   {'year': '2018',
    'period': 'M08',
    'periodName': 'August',
    'value': '252.146',
    'footnotes': [{}]},
   {'year': '2018',
    'period': 'M07',
    'periodName': 'July',
    'value': '252.006',
    'footnotes': [{}]},
   {'year': '2018',
    'period': 'M06',
    'periodName': 'June',
    'value': '251.989',
    'footnotes': [{}]},
   {'year': '2018',
    'period': 'M05',
    'periodName': 'May',
    'value': '251.588',
    'footnotes': [{}]},
   {'year': '2018',
    'p

In [20]:
type(lst)

list

In [23]:
agent = create_openai_tools_agent(tools=[load_indicator],llm=llm,prompt=prompt)
executor = AgentExecutor(agent=agent,tools=[load_indicator],verbose=True)

In [25]:
for element in map_indicator:
    print(map_indicator.get(element))
    input_user=f"Pull {element} data for 2017"
    #print(input)
    response = executor.invoke({"input":input_user})
    print(response['output'])

CUUR0000SA0


> Entering new AgentExecutor chain...

Invoking: `load_indicator` with `{'indicator': 'CPI_for_All_Urban_Consumers_CPIU_Unadjusted', 'start': '2017', 'end': '2017'}`


[{'seriesID': 'CUUR0000SA0', 'data': [{'year': '2017', 'period': 'M12', 'periodName': 'December', 'value': '246.524', 'footnotes': [{}]}, {'year': '2017', 'period': 'M11', 'periodName': 'November', 'value': '246.669', 'footnotes': [{}]}, {'year': '2017', 'period': 'M10', 'periodName': 'October', 'value': '246.663', 'footnotes': [{}]}, {'year': '2017', 'period': 'M09', 'periodName': 'September', 'value': '246.819', 'footnotes': [{}]}, {'year': '2017', 'period': 'M08', 'periodName': 'August', 'value': '245.519', 'footnotes': [{}]}, {'year': '2017', 'period': 'M07', 'periodName': 'July', 'value': '244.786', 'footnotes': [{}]}, {'year': '2017', 'period': 'M06', 'periodName': 'June', 'value': '244.955', 'footnotes': [{}]}, {'year': '2017', 'period': 'M05', 'periodName': 'May', 'value': '244.733', 'footnotes': [{